# Análisis de imágen con Python

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from sklearn.cluster import KMeans
from scipy import ndimage
import skimage as skim
from skimage import color
from skimage import io
from skimage.color import label2rgb
from skimage.measure import regionprops
import pandas as pd
import glob
import os

In [2]:
archivos = glob.glob('**/**/*.jpg')
#archivos = glob.glob('5°C-3H/SALATO/*.jpg')

nombres = []
i = 0
for archivo in archivos:
#    file = os.path.basename(archivos[i])
    file = archivos[i]
    file1 = file.replace('.jpg','')
    nombres.append(file1)
    i = i+1
df = pd.DataFrame(columns = ['Muestra', 'L', 'a', 'b'])  # DataFrame donde se almacenarán los valores de color
j=0
for archivo in archivos:
    image = skim.io.imread(archivo)  # abro el archivo de imagen
    labImage = color.rgb2lab(image, illuminant='D65', observer='2')  # img_Lab es un array de numpy de 3 dimensiones (L, a, b)
    r = image[:, :, 0];             # red channel
    g = image[:, :, 1];             # green channel
    b = image[:, :, 2];             # blue channel
    L = labImage[:, :, 0];            
    A = labImage[:, :, 1];             
    B = labImage[:, :, 2];
    rgbMask = r > 100;
    lMask = L < 65;
    aMask = A > 10;
    bMask = B > 10;
    red = r * rgbMask * lMask * aMask * bMask;
    green = g * rgbMask * lMask * aMask * bMask;
    blue = b * rgbMask * lMask * aMask * bMask;
    rgb = (red[..., np.newaxis], green[..., np.newaxis], blue[..., np.newaxis])
    rgbImage1 = np.concatenate(rgb, axis=-1)
    labImage1 = color.rgb2lab(rgbImage1, illuminant='D65', observer='2')  # Convierte cada imagen a L a b
    L = np.round(np.nanmean(np.where(labImage1[:,:,0]!=0,labImage1[:,:,0],np.nan)),decimals=2)  # Calcula el promedio de L en la primer matriz sin tener en cuenta los 0
    a = np.round(np.nanmean(np.where(labImage1[:,:,1]!=0,labImage1[:,:,1],np.nan)),decimals=2)  # porque los convierte en NaN y la función promedio no los considera
    b = np.round(np.nanmean(np.where(labImage1[:,:,2]!=0,labImage1[:,:,2],np.nan)),decimals=2)  # además redondea los valores obtenidos a 2 decimales
    nueva_fila = {'Muestra':nombres[j], 'L':L, 'a':a, 'b':b} # Diccionario donde almacena los valores calculados en cada Loop
    df = df.append(nueva_fila, ignore_index=True)  # Agrega los valores calculados al DataFrame inicial
    j = j + 1
df.to_excel('tabla_color.xlsx',index=False)  # Guarda la tabla de colores en un archivo excel
df

,Muestra,L,a,b
0,45°C-3h/FRESCO/667,60.05,34.34,38.79
1,45°C-3h/FRESCO/659,59.50,38.10,41.31
2,45°C-3h/FRESCO/676,60.42,35.92,40.57
3,45°C-3h/FRESCO/668,59.32,37.87,40.86
4,45°C-3h/FRESCO/621,59.17,40.04,42.34
5,45°C-3h/salato/667,52.91,40.40,41.60
6,45°C-3h/salato/659,53.23,40.82,40.45
7,45°C-3h/salato/676,54.22,40.87,41.56
8,45°C-3h/salato/668,54.09,41.59,40.80
9,45°C-3h/salato/621,52.52,42.69,41.78


In [3]:
len(archivos)

60